In [1]:
import xcast as xc 
import xarray as xr 
import cartopy.crs as ccrs 

model = xr.open_dataset('data/MODEL_JJA.nc').prec
obs = xr.open_dataset('data/OBS_JJA.nc').precip.expand_dims({'M':[0]})
# we switch model and obs to regrid the obs to the model grid, rather than model to obs grid
obs, model = xc.match(obs, model) 

In [2]:
model = model.sel(longitude=slice(-120, -70), latitude=slice(20,60))
obs = obs.sel(longitude=slice(-120, -70), latitude=slice(20,60))

drymask = xc.drymask(obs, dry_threshold=30, quantile_threshold=0.3)
obs = obs*drymask 
model = model * drymask

In [ ]:
gene_set = {
    'regularization': [i for i in range(-20, 20) ],
    'hidden_layer_size': [i for i in range(3, 16)],#, 80, 160, 320, 640],
    'activation': ['relu', 'sigm', 'tanh', 'lin', 'leaky', 'elu', 'softplus'],
    'preprocessing': ['minmax', 'std', 'none' ],
    'encoding': [  'nonexceedance', 'binary'],
    'quantiles': [ [1/3,2/3], [0.0001, 0.5, 0.9999], [0.2, 0.4, 0.6, 0.8], [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9] ],
    'standardize_y': [True, False],
    'n_estimators': [5],
    'initialization': ['uniform', 'normal', 'xavier']
}

reg = xc.EPOELM()
params = reg.tune(model, obs, queue_len=4, generation_size=2, **gene_set)


In [ ]:
params.goodness.plot(vmin=-1)

In [ ]:
xc.save_parameters(params.params, dest='test.params')

In [ ]:
params2 = xc.load_parameters('test.params')

In [ ]:
hindcasts_det, hindcasts_prob = [], []
i=1
for xtrain, ytrain, xtest, ytest in xc.CrossValidator(model, obs, window=3):
    print("window {}".format(i))
    i += 1
    epoelm = xc.EPOELM(params=params2)
    epoelm.fit(xtrain, ytrain)
    probs =  epoelm.predict_proba(xtest)
    preds = epoelm.predict(xtest)
    hindcasts_det.append(preds.isel(time=1))
    hindcasts_prob.append(probs.isel(time=1))
hindcasts_det = xr.concat(hindcasts_det, 'time')
hindcasts_prob = xr.concat(hindcasts_prob, 'time')


In [ ]:
ohc = xc.OneHotEncoder() 
ohc.fit(obs)
T = ohc.transform(obs)
clim = xr.ones_like(T) * 0.333

In [ ]:
clim_rps = xc.RankProbabilityScore(clim, T)
pred_rps = xc.RankProbabilityScore(hindcasts_prob, T)
rpss = 1 - pred_rps / clim_rps 

In [ ]:
pl = rpss.plot(subplot_kws={'projection': ccrs.PlateCarree()})
c = pl.axes.coastlines()

In [ ]:
pearson = xc.Pearson(hindcasts_det, obs)

In [ ]:
pl = pearson.plot(subplot_kws={'projection': ccrs.PlateCarree()})
c = pl.axes.coastlines()